In [ ]:
#pip install selenium
#To scrape the unique ids

In [1]:
import json
import pandas as pd

from bs4 import BeautifulSoup
from requests import get
import requests

from selenium import webdriver
import time

from random import randint
from time import sleep

import pprint
pp = pprint.PrettyPrinter(indent =4)

In [ ]:
#This is to flatten nested list
import itertools

def oneDArray(x):
    return list(itertools.chain(*x))

In [2]:
Review = pd.read_json('reviews_Beauty_5.json',lines = True)

In [3]:
Review.head(100)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
95,ASXPNU896V9I8,B00004U9UY,L90,"[0, 0]","I have dry hands, as my job requires me to was...",5,Best hand lotion,1360800000,"02 14, 2013"
96,A3S3R88HA0HZG3,B00004U9UY,PT Cruiser,"[1, 1]",This hand cream has one of the nicest fragranc...,5,An effective hand cream with a wonderful scent,1238544000,"04 1, 2009"
97,A2SX3JU39YE2TP,B00004U9UY,Summer Haven,"[0, 0]",I first purchased this hand treatment from a g...,4,"Wonderful feel, wonderful fragrance",1374192000,"07 19, 2013"
98,AD526SECC3I9D,B00004U9V2,Anuradha J. Cetta,"[0, 0]","Best hand cream I've ever used, and I NEVER us...",5,Great cream!,1386201600,"12 5, 2013"


In [4]:
type(Review)

pandas.core.frame.DataFrame

# EDA and Scraping


In [5]:
#Number of data points
Review.shape
#There are 198502 data points

(198502, 9)

In [6]:
#Finding the unique products
len(Review['asin'].unique())
#There are 12101 unique products

12101

In [7]:
#Checking the number of unique users
len(Review['reviewerID'].unique())
#There are 22363 unique user

22363

In [8]:
#columns
Review.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [9]:
Review.head(50)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
5,A3BTN14HIZET6Z,7806397051,"S. M. Randall ""WildHorseWoman""","[1, 2]","I was very happy to get this palette, now I wi...",5,Very nice palette!,1365984000,"04 15, 2013"
6,A1Z59RFKN0M5QL,7806397051,"tasha ""luvely12b""","[1, 3]",PLEASE DONT DO IT! this just rachett the palet...,1,smh!!!,1376611200,"08 16, 2013"
7,AWUO9P6PL1SY8,7806397051,TreMagnifique,"[0, 1]","Chalky,Not Pigmented,Wears off easily,Not a Co...",2,"Chalky, Not Pigmented, Wears off easily, Not a...",1378252800,"09 4, 2013"
8,A3LMILRM9OC3SA,9759091062,NaN,"[0, 0]",Did nothing for me. Stings when I put it on. I...,2,"no Lightening, no Brightening,......NOTHING",1405209600,"07 13, 2014"
9,A30IP88QK3YUIO,9759091062,Amina Bint Ibraheem,"[0, 0]",I bought this product to get rid of the dark s...,3,Its alright,1388102400,"12 27, 2013"


The columns are:
1. reviewerID: This column gives the unique identity to each amazon user.
2 asin: This column gives the product number
3. helpful: List L, where L[0]: Number of people who found it helpful, L[1]: Number of people who did not find it helpful
4.reviewText: Complete descriotion of the review
5.overall: overall rating of the product
6.Summary: brief into to the review
7.unixReviewTime: Time of review
8. reviewTime: Date of Review

The asin column will be used to scrape additional data from the current amazon product website
A list containing unique asin id is formed which will be used to iterate during web scraping


In [10]:
unique_asin =[]
unique_asin = list(Review['asin'].unique())

In [11]:
type(unique_asin)

list

In [13]:
unique_asin[:10]

['7806397051',
 '9759091062',
 '9788072216',
 '9790790961',
 '9790794231',
 'B00004TMFE',
 'B00004TUBL',
 'B00004TUBV',
 'B00004U9UY',
 'B00004U9V2']

The unique_asin list will be iterated through to enter the product page and scrape for more information. Due to block from anti robot pages, fields with unavailable rows will be rescraped using a brand new driver object and then visiting the same url again. 
The url of the product page is of the format: https://www.amazon.com/gp/product/{asin id}

# # #Web Scraping

In [12]:
#The file did not have updated product name, color and description, we will scrape it from amazon website
import selenium
from selenium import webdriver

In [17]:
#Instantiating the web driver
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")

In [18]:
#Creating an empty list to store dictionary for each product
list_of_products = []

The unique_asin list is first iterated and tags are located to scrape the following data
1. Product name
2. Price
3. Rating
4. Number of Rating
5. Category of beauty product
6. Availability

In [19]:
#Instantiating the web driver for product with asin B00004TMFE
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
driver.get(f'https://www.amazon.com/gp/product/B00004TMFE')

# Instantiate BeautifulSoup web scraper 
soup = BeautifulSoup(driver.page_source, 'lxml') 

#dictionary to store the key-value pair
prod1_dict = {}

# Save product ID 
prod1_dict['asin'] = 'B00004TMFE'

# Look for product title 
prod1_dict['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
# Look for product categories list 
prod1_dict['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
        
# Look for product description
desc = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
prod1_dict['description'] =" ".join(desc.split())
        
#Look for Product Price
prod1_dict['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
#Look for Rating
prod1_dict['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
#Number of Ratings
prod1_dict['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()

pp.pprint(prod1_dict)
                            

{   'No_of_Rating': '5,646 ratings',
    'asin': 'B00004TMFE',
    'category': 'Hair Care',
    'description': 'Brand Avalon Organics Scent Biotin B Hair Type Thin Liquid '
                   'Volume 14 Fluid Ounces Ingredients Aloe Barbadensis Leaf '
                   'Juice (Certified Organic Ingredient), Aqua (Water), '
                   'Brassica Alcohol, Glycerin, Cocos Nucifera (Coconut) Oil '
                   '(Certified Organic Ingredient), Brassicyl Isoleucinate '
                   'Esylate, Gluconolactone, Capryloyl Glycerin/Sebacic Acid '
                   'Copolymer, Cedrus Atlantica Wood Oil, Citrus Aurantifolia '
                   '(Lime) Oil, Citrus Aurantium Dulcis (Orange) Peel Oil, '
                   'Citrus Limon (Lemon) Peel Oil, Citrus Paradisi '
                   '(Grapefruit) Peel Oil, Eucalyptus Globulus Leaf Oil, '
                   'Juniperus Virginiana Oil, Persea Gratissima (Avocado) Oil '
                   '(Certified Organic Ingredient), Rosmar

# Scraping First 10 Products

In [1]:
list_of_products_10 = []
list_of_unavailable_products_10 =[]

#Instantiating the web driver(product 1-10)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")

#Start looping through first 500 products in the asin list
for i in unique_asin[:10]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
        
        # Look for product description
        desc = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        product_info['description']=" ".join(desc.split())
        
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10.append(product_info)
        

    except:
        list_of_unavailable_products_10.append(i)
        print('not available',i)
        

In [64]:
list_of_unavailable_products_10

['7806397051',
 '9759091062',
 '9790790961',
 '9790794231',
 'B00004U9UY',
 'B00004U9V2']

In [22]:
print(len(list_of_products_10))
print(len(list_of_unavailable_products_10))

4
6


In [65]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_10 = pd.DataFrame(list_of_products_10)

list_of_products_10.to_csv('list_of_products_10.csv')

In [66]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_10 = pd.DataFrame(list_of_unavailable_products_10)

list_of_unavailable_products_10.to_csv('list_of_unavailable_products_10.csv')

In [67]:
list_of_unavailable_products_10=list_of_unavailable_products_10.values.tolist()

In [72]:
list_of_unavailable_products_10

[['7806397051'],
 ['9759091062'],
 ['9790790961'],
 ['9790794231'],
 ['B00004U9UY'],
 ['B00004U9V2']]

In [74]:
oneDArray(list_of_unavailable_products_10)

['7806397051',
 '9759091062',
 '9790790961',
 '9790794231',
 'B00004U9UY',
 'B00004U9V2']

In [25]:
#!cat 'list_of_products_10.csv'

In [26]:
pp.pprint(list_of_products_10)

         asin                                               name   category  \
0  9788072216  Prada Candy by Prada for Women 1.7 oz Eau de P...  Fragrance   
1  B00004TMFE  Avalon Organics Therapy Thickening Conditioner...  Hair Care   
2  B00004TUBL  CLASSIC Better Living Two Chamber Dispenser, W...       Bath   
3  B00004TUBV  Better Living Products 73350 ULTI-MATE 3 Chamb...       Bath   

                                         description   price  \
0  Brand Prada Scent Honey, Musk , Vanilla Item F...  $56.29   
1  Brand Avalon Organics Scent Biotin B Hair Type...   $7.81   
2  Color White Brand CLASSIC Item Dimensions LxWx...  $39.99   
3  Color White Material Plastic Brand Better Livi...  $38.21   

               rating   No_of_Rating  
0  4.7 out of 5 stars  1,391 ratings  
1  4.1 out of 5 stars  5,646 ratings  
2  4.3 out of 5 stars     99 ratings  
3  4.5 out of 5 stars  1,245 ratings  


# #Scrape Products 1-2000

In [2]:
list_of_products_2000 = []
list_of_unavailable_products_2000 =[]

#Instantiating the web driver(product 1-500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")

#Start looping through first 500 products in the asin list
for i in unique_asin[:500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
       
        # Look for product description
        desc = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        product_info['description']=" ".join(desc.split())
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_2000.append(product_info)
        

    except:
        list_of_unavailable_products_2000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 500-1000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")


#Start looping through 500-1000 products in the asin list
for i in unique_asin[500:1000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_2000.append(product_info)
        

    except:
        list_of_unavailable_products_2000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 1000-15000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 1000-1500 products in the asin list
for i in unique_asin[1000:1500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_2000.append(product_info)
        

    except:
        list_of_unavailable_products_2000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 1500-2000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 1000-1500 products in the asin list
for i in unique_asin[1500:2000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_2000.append(product_info)
        

    except:
        list_of_unavailable_products_2000.append(i)
        print('not available',i)


        

In [28]:
print(len(list_of_products_2000))
print(len(list_of_unavailable_products_2000))

846
1154


In [29]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_2000 = pd.DataFrame(list_of_products_2000)

list_of_products_2000.to_csv('list_of_products_2000.csv')

In [30]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_2000 = pd.DataFrame(list_of_unavailable_products_2000)

list_of_unavailable_products_2000.to_csv('list_of_unavailable_products_2000.csv')

## Scrape Products 2000-4000 

In [4]:
list_of_products_4000 = []
list_of_unavailable_products_4000 =[]


#Instantiating the web driver(product 2000-2500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 2000-2500 products in the asin list
for i in unique_asin[2000:2500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_4000.append(product_info)
        

    except:
        list_of_unavailable_products_4000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 2500-3000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 2500-3000 products in the asin list
for i in unique_asin[2500:3000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_4000.append(product_info)
        

    except:
        list_of_unavailable_products_4000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 3000-35000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 3000-3500 products in the asin list
for i in unique_asin[3000:3500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_4000.append(product_info)
        

    except:
        list_of_unavailable_products_4000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 3500-4000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 3500-4000 products in the asin list
for i in unique_asin[3500:4000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_4000.append(product_info)
        

    except:
        list_of_unavailable_products_4000.append(i)
        print('not available',i)
        

In [32]:
print(len(list_of_products_4000))
print(len(list_of_unavailable_products_4000))

802
1198


In [33]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_4000 = pd.DataFrame(list_of_products_4000)

list_of_products_4000.to_csv('list_of_products_4000.csv')

In [34]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_4000 = pd.DataFrame(list_of_unavailable_products_4000)

list_of_unavailable_products_4000.to_csv('list_of_unavailable_products_4000.csv')

# #Scrape Products 4000-6000 

In [5]:
list_of_products_6000 = []
list_of_unavailable_products_6000 =[]

#Instantiating the web driver(product 4000-4500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 4000-4500 products in the asin list
for i in unique_asin[4000:4500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_6000.append(product_info)
        

    except:
        list_of_unavailable_products_6000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 4500-5000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 4500-5000 products in the asin list
for i in unique_asin[4500:5000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_6000.append(product_info)
        

    except:
        list_of_unavailable_products_6000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 5000-55000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 5000-5500 products in the asin list
for i in unique_asin[5000:5500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_6000.append(product_info)
        

    except:
        list_of_unavailable_products_6000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 5500-6000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 5500-6000 products in the asin list
for i in unique_asin[5500:6000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_6000.append(product_info)
        

    except:
        list_of_unavailable_products_6000.append(i)
        print('not available',i)

     

In [36]:
print(len(list_of_products_6000))
print(len(list_of_unavailable_products_6000))

781
1219


In [37]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_6000 = pd.DataFrame(list_of_products_6000)

list_of_products_6000.to_csv('list_of_products_6000.csv')

In [38]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_6000 = pd.DataFrame(list_of_unavailable_products_6000)

list_of_unavailable_products_6000.to_csv('list_of_unavailable_products_6000.csv')

# Scrape Products 6000-8000

In [6]:
list_of_products_8000 = []
list_of_unavailable_products_8000 =[]


#Instantiating the web driver(product 6000-6500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 6000-6500 products in the asin list
for i in unique_asin[6000:6500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_8000.append(product_info)
        

    except:
        list_of_unavailable_products_8000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 6500-7000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 6500-7000 products in the asin list
for i in unique_asin[6500:7000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_8000.append(product_info)
        

    except:
        list_of_unavailable_products_8000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 7000-75000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 7000-7500 products in the asin list
for i in unique_asin[7000:7500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_8000.append(product_info)
        

    except:
        list_of_unavailable_products_8000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 7500-8000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 7500-8000 products in the asin list
for i in unique_asin[7500:8000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_8000.append(product_info)
        

    except:
        list_of_unavailable_products_8000.append(i)
        print('not available',i)

        

In [40]:
print(len(list_of_products_8000))
print(len(list_of_unavailable_products_8000))

654
1346


In [41]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_8000 = pd.DataFrame(list_of_products_8000)

list_of_products_8000.to_csv('list_of_products_8000.csv')

In [42]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_8000 = pd.DataFrame(list_of_unavailable_products_8000)

list_of_unavailable_products_8000.to_csv('list_of_unavailable_products_8000.csv')

# Scraping Products 8000-10000

In [7]:

list_of_products_10000 = []
list_of_unavailable_products_10000 =[]

#Instantiating the web driver(product 8000-8500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 8000-8500 products in the asin list
for i in unique_asin[8000:8500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10000.append(product_info)
        

    except:
        list_of_unavailable_products_10000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 8500-9000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 8500-9000 products in the asin list
for i in unique_asin[8500:9000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10000.append(product_info)
        

    except:
        list_of_unavailable_products_10000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 7000-75000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 9000-9500 products in the asin list
for i in unique_asin[9000:9500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10000.append(product_info)
        

    except:
        list_of_unavailable_products_10000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 9500-10000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 9500-10000 products in the asin list
for i in unique_asin[9500:10000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10000.append(product_info)
        

    except:
        list_of_unavailable_products_10000.append(i)
        print('not available',i)


In [44]:
print(len(list_of_products_10000))
print(len(list_of_unavailable_products_10000))

567
1433


In [45]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_10000 = pd.DataFrame(list_of_products_10000)

list_of_products_10000.to_csv('list_of_products_10000.csv')

In [46]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_10000 = pd.DataFrame(list_of_unavailable_products_10000)

list_of_unavailable_products_10000.to_csv('list_of_unavailable_products_10000.csv')

# # Scraping products 10000-12101

In [8]:
list_of_products_12000 = []
list_of_unavailable_products_12000 =[]


#Instantiating the web driver(product 10000-10500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in unique_asin[10000:10500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_12000.append(product_info)
        

    except:
        list_of_unavailable_products_12000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 10500-11000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 10500-11000 products in the asin list
for i in unique_asin[10500:11000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_12000.append(product_info)
        

    except:
        list_of_unavailable_products_12000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 11000-115000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 11000-11500 products in the asin list
for i in unique_asin[11000:11500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_12000.append(product_info)
        

    except:
        list_of_unavailable_products_12000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 11500-12101)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 11500-12101 products in the asin list
for i in unique_asin[11500:12101]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_12000.append(product_info)
        

    except:
        list_of_unavailable_products_12000.append(i)
        print('not available',i)


In [50]:
print(len(list_of_products_12000))
print(len(list_of_unavailable_products_12000))

509
1592


In [51]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_12000 = pd.DataFrame(list_of_products_12000)

list_of_products_12000.to_csv('list_of_products_12000.csv')

In [52]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_12000 = pd.DataFrame(list_of_unavailable_products_12000)

list_of_unavailable_products_12000.to_csv('list_of_unavailable_products_12000.csv')

Scraping the unavailable columns to check for anti robot block

In [59]:
type(list_of_unavailable_products_2000)

list

In [58]:
list_of_unavailable_products_2000=list_of_unavailable_products_2000.values.tolist()

In [62]:
list_of_unavailable_products_2000[:10]

[['7806397051'],
 ['9759091062'],
 ['9790790961'],
 ['9790794231'],
 ['B00004U9UY'],
 ['B00004U9V2'],
 ['B000050B6U'],
 ['B000052WY7'],
 ['B000052WYL'],
 ['B000052XZP']]

In [75]:
list_of_unavailable_products_2000=oneDArray(list_of_unavailable_products_2000)

In [76]:
list_of_unavailable_products_2000[:10]

['7806397051',
 '9759091062',
 '9790790961',
 '9790794231',
 'B00004U9UY',
 'B00004U9V2',
 'B000050B6U',
 'B000052WY7',
 'B000052WYL',
 'B000052XZP']

In [9]:

list_of_available_products_Rerun_2000 = []
list_of_unavailable_products_Rerun_2000 =[]


#Instantiating the web driver(product 10000-10500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in list_of_unavailable_products_2000:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_available_products_Rerun_2000.append(product_info)
        

    except:
        list_of_unavailable_products_Rerun_2000.append(i)
        print('not available',i)
        


In [79]:
print(len(list_of_available_products_Rerun_2000))
print(len(list_of_unavailable_products_Rerun_2000))

31
1123


In [83]:
#Change the list to pandas dataframe to write onto a csv file
list_of_available_products_Rerun_2000 = pd.DataFrame(list_of_available_products_Rerun_2000)

list_of_available_products_Rerun_2000.to_csv('list_of_available_products_Rerun_2000.csv')

In [84]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_Rerun_2000 = pd.DataFrame(list_of_unavailable_products_Rerun_2000)

list_of_unavailable_products_Rerun_2000.to_csv('list_of_unavailable_products_Rerun_2000.csv')

In [82]:
type(list_of_available_products_Rerun_2000)

pandas.core.frame.DataFrame

In [89]:
list_of_unavailable_products_4000[:10]

['B000PHRIQM',
 'B000PHVNE0',
 'B000PHZ26K',
 'B000PHZE7M',
 'B000PI3BS0',
 'B000PI4J3G',
 'B000PI57HS',
 'B000PI9MO2',
 'B000PK4XRQ',
 'B000PKUYYW']

In [87]:
list_of_unavailable_products_4000=list_of_unavailable_products_4000.values.tolist()

In [88]:
list_of_unavailable_products_4000=oneDArray(list_of_unavailable_products_4000)

In [10]:
list_of_available_products_Rerun_4000 = []
list_of_unavailable_products_Rerun_4000 =[]


#Instantiating the web driver
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in list_of_unavailable_products_4000:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_available_products_Rerun_4000.append(product_info)
        

    except:
        list_of_unavailable_products_Rerun_4000.append(i)
        print('not available',i)
        


In [91]:
print(len(list_of_available_products_Rerun_4000))
print(len(list_of_unavailable_products_Rerun_4000))

40
1158


In [92]:
#Change the list to pandas dataframe to write onto a csv file
list_of_available_products_Rerun_4000 = pd.DataFrame(list_of_available_products_Rerun_4000)

list_of_available_products_Rerun_4000.to_csv('list_of_available_products_Rerun_4000.csv')

In [93]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_Rerun_4000 = pd.DataFrame(list_of_unavailable_products_Rerun_4000)

list_of_unavailable_products_Rerun_4000.to_csv('list_of_unavailable_products_Rerun_4000.csv')

In [94]:
list_of_unavailable_products_6000=list_of_unavailable_products_6000.values.tolist()

In [95]:
list_of_unavailable_products_6000=oneDArray(list_of_unavailable_products_6000)

In [11]:
list_of_available_products_Rerun_6000 = []
list_of_unavailable_products_Rerun_6000 =[]


#Instantiating the web driver
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in list_of_unavailable_products_6000:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_available_products_Rerun_6000.append(product_info)
        

    except:
        list_of_unavailable_products_Rerun_6000.append(i)
        print('not available',i)
        


In [97]:
print(len(list_of_available_products_Rerun_6000))
print(len(list_of_unavailable_products_Rerun_6000))

12
1207


In [98]:
#Change the list to pandas dataframe to write onto a csv file
list_of_available_products_Rerun_6000 = pd.DataFrame(list_of_available_products_Rerun_6000)

list_of_available_products_Rerun_6000.to_csv('list_of_available_products_Rerun_6000.csv')

In [99]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_Rerun_6000 = pd.DataFrame(list_of_unavailable_products_Rerun_6000)

list_of_unavailable_products_Rerun_6000.to_csv('list_of_unavailable_products_Rerun_6000.csv')

In [100]:
list_of_unavailable_products_8000=list_of_unavailable_products_8000.values.tolist()

In [101]:
list_of_unavailable_products_8000=oneDArray(list_of_unavailable_products_8000)

In [12]:
list_of_available_products_Rerun_8000 = []
list_of_unavailable_products_Rerun_8000 =[]


#Instantiating the web driver
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in list_of_unavailable_products_8000:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_available_products_Rerun_8000.append(product_info)
        

    except:
        list_of_unavailable_products_Rerun_8000.append(i)
        print('not available',i)
        


In [ ]:
print(len(list_of_available_products_Rerun_8000))
print(len(list_of_unavailable_products_Rerun_8000))

In [ ]:
#Change the list to pandas dataframe to write onto a csv file
list_of_available_products_Rerun_8000 = pd.DataFrame(list_of_available_products_Rerun_8000)

list_of_available_products_Rerun_8000.to_csv('list_of_available_products_Rerun_8000.csv')

In [ ]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_Rerun_8000 = pd.DataFrame(list_of_unavailable_products_Rerun_8000)

list_of_unavailable_products_Rerun_8000.to_csv('list_of_unavailable_products_Rerun_8000.csv')